## Ques. Develop a RNN model for analysis the sentiment of the tweets using Sentiment140 dataset”

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
!unzip /content/drive/MyDrive/training.1600000.processed.noe moticon.csv



unzip:  cannot find or open /content/drive/MyDrive/training.1600000.processed.noe, /content/drive/MyDrive/training.1600000.processed.noe.zip or /content/drive/MyDrive/training.1600000.processed.noe.ZIP.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/training.1600000.processed.noemoticon.csv',names=["Sentiment", "ID", "Date", "Query", "User", "Tweet"],encoding='latin-1')
df.head()

,Sentiment,ID,Date,Query,User,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
X = df.Tweet
Y = df.Sentiment
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

Split into training and test data.

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [ ]:
from keras.utils import pad_sequences

max_words = 100
max_len = 128
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

### RNN
Define the RNN structure.

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(1024,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 128)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 128, 50)           5000      
                                                                 
 lstm_2 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 1024)              66560     
                                                                 
 activation_4 (Activation)   (None, 1024)              0         
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 out_layer (Dense)           (None, 1)                 1025

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=1024,epochs=5,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/5
1063/1063 [==============================] - 1265s 1s/step - loss: 0.6088 - accuracy: 0.4998 - val_loss: 0.6038 - val_accuracy: 0.4995
Epoch 2/5
1063/1063 [==============================] - 1168s 1s/step - loss: 0.6002 - accuracy: 0.4998 - val_loss: 0.5955 - val_accuracy: 0.4995
Epoch 3/5
1063/1063 [==============================] - ETA: 0s - loss: 0.5953 - accuracy: 0.4998

Process the test set data.

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

Evaluate the model on the test set.

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

7500/7500 [==============================] - 188s 25ms/step - loss: 0.6933 - accuracy: 0.5015


In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.693
  Accuracy: 0.501
